# 数据产品设计-2

## 数据准备

In [1]:
import streamlit as st
from streamlit_echarts import st_echarts
from pyecharts import options as opts
from pyecharts.charts import Bar
from streamlit_echarts import st_pyecharts
import pandas as pd
import numpy as np

2023-03-22 09:17:43.802 INFO    numexpr.utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-03-22 09:17:43.806 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
a=pd.read_csv(r"D:\try\shoes.csv")
a.sales=a.sales.str.replace("人付款","").astype(int)
#求出各个商品的销售额并把它并入到原始数据框中去
z1=a.sales*a.price
z1.name="xse"
a1=pd.concat([a,z1],axis=1)#给序列命名之后添加入数据框就会直接以序列名作为列标

In [3]:
#先做成字典，把各个特征放入字典中
te_zheng={"nick":[],"z_xse":[],"z_num":[],"p_sales":[],"p_bdj":[],"p_price":[]}
for i in a1.groupby("nick"):
    te_zheng["nick"].append(i[0])
    te_zheng["z_xse"].append(i[1].xse.sum())
    te_zheng["z_num"].append(len(i[1]))
    te_zheng["p_sales"].append(round(i[1].sales.mean(),1))
    if i[1].sales.sum()==0:#存在除零的情况，所以做判断
        te_zheng["p_bdj"].append(0)
    else:
        te_zheng["p_bdj"].append(round(i[1].xse.sum()/i[1].sales.sum(),1))
    te_zheng["p_price"].append(round(i[1].price.mean(),1))

In [4]:
# 把字典转化为数据框，并基于销售额排序
df_te_zheng=pd.DataFrame(te_zheng)
df_te_zheng.sort_values(by="z_xse",ascending=False,inplace=True)
df_te_zheng.set_index("nick",inplace=True)
df_te_zheng.head()

,z_xse,z_num,p_sales,p_bdj,p_price
nick,,,,,
意尔康皮鞋旗舰店,3558446.0,289,47.8,257.4,485.7
意尔康男鞋旗舰店,792685.0,149,18.0,296.3,680.8
意尔康品牌店,114452.0,65,7.7,229.8,271.5
乙方乙方88888,88144.0,9,60.3,162.3,164.7
吸引力xl,43870.0,5,59.0,148.7,152.0


## 页面的布局

* Streamlit 可以使用 st.sidebar 在左侧面板侧边栏中轻松组织小部件。 传递给 st.sidebar 的每个元素都固定在左侧，允许用户专注于应用程序中的内容，同时仍然可以访问 UI 控件。

* st.columns 允许您并排放置小部件

In [ ]:
# Add a selectbox to the sidebar:
add_selectbox = st.sidebar.selectbox(
    'How would you like to be contacted?',
    ('Email', 'Home phone', 'Mobile phone')
)

# Add a slider to the sidebar:
add_slider = st.sidebar.slider(
    'Select a range of values',
    0.0, 100.0, (25.0, 75.0)
)

In [ ]:
left_column, right_column = st.columns(2)
# You can use a column just like st.sidebar:
left_column.button('Press me!')

# Or even better, call Streamlit functions inside a "with" block:
with right_column:
    chosen = st.radio(
        'Sorting hat',
        ("Gryffindor", "Ravenclaw", "Hufflepuff", "Slytherin"))
    st.write(f"You are in {chosen} house!")

In [ ]:
col1, col2 = st.columns([3, 1])
data = np.random.randn(10, 1)

col1.subheader("A wide column with a chart")
col1.line_chart(data)

col2.subheader("A narrow column with the data")
col2.write(data)

## 多媒体的展现

https://docs.streamlit.io/library/api-reference/media

* st.image

* st.audio

* st.video

In [ ]:
import streamlit as st
from PIL import Image

image = Image.open('sunrise.jpg')

st.image(image, caption='Sunrise by the mountains')
st.image("http://abs.hznu.edu.cn/upload/resources/image/2022/07/31/7729995.jpg", caption='Sunrise by the mountains')

In [ ]:
video_file = open('myvideo.mp4', 'rb')
video_bytes = video_file.read()

st.video(video_bytes)

In [2]:
3/8

0.375

In [4]:
4*3/(8*8)

0.1875

In [5]:
0.375/0.1875

2.0

## 特殊数字的展示

* st.metric

https://docs.streamlit.io/library/api-reference/data/st.metric

In [ ]:
st.metric(label="Temperature", value="70 °F", delta="1.2 °F")

col1, col2, col3 = st.columns(3)
col1.metric("Temperature", "70 °F", "1.2 °F")
col2.metric("Wind", "9 mph", "-8%")
col3.metric("Humidity", "86%", "4%")

st.metric(label="Gas price", value=4, delta=-0.5,
    delta_color="inverse")

st.metric(label="Active developers", value=123, delta=123,
    delta_color="off")

## classwork 1 (布局的初步应用)

1，请将上节课作业中的组件移入侧边栏

* st.image

2，请找两张图片，并排显示在页面上

* 特殊数值展示（st.metric）-以粗体大字体显示数值。

3, 拷贝课件的代码显示各个店铺的特征数据框

4，选择某个店铺的特征并通过st.metric并排展示在页面上

In [ ]:
#这是上节课作业代码
a=pd.read_csv(r"D:\try\shoes.csv")
a.sales=a.sales.str.replace("人付款","").astype(int)


option4 = st.multiselect(
    'What are your favorite shopes',
    a.groupby("nick").size().sort_values(ascending=False).index,
    ['意尔康男鞋旗舰店'])

p1=a.groupby('info.鞋面材质').size().sort_values()
p1.name="all"

chart0 = {
  "yAxis": {
    "type": 'category',
    "data": p1.index.tolist()
  },
  "xAxis": {
    "type": 'value'
  },
  "legend": {},
  "series": [
      {
        "data": p1.values.tolist(),
        "type": 'bar',
        "name":"all"
      },
  ]
}

for i in option4:
    p2=a[a["nick"]==i].groupby('info.鞋面材质').size()
    p2.name=i
    p1=pd.concat([p1,p2],axis=1).fillna(0)
    chart0["series"].append({
      "data": p1[i].values.tolist(),
      "type": 'bar',
      "name": i
    })

st_echarts(chart0)

p1

option0 = st.selectbox(
    '选择鞋面材质',
     a["info.鞋面材质"].unique())
option1 = st.slider('price',0.,a.price.max()/4,(100.,300.))
option2 = st.slider('sales',0,a.sales.max(),(100,300))

option3 = st.multiselect(
    'What are your favorite colors',
    a.columns,
    ['nick','title','price', 'sales'])

a1=a[(a["info.鞋面材质"]==option0)&(a.price>option1[0])&(a.price<option1[1])
     &(a.sales>option2[0])&(a.sales<option2[1])][option3]

a1

In [6]:
df_te_zheng.head()

,nick,z_xse,z_num,p_sales,p_bdj,p_price
285,意尔康皮鞋旗舰店,3558446.0,289,47.8,257.4,485.7
284,意尔康男鞋旗舰店,792685.0,149,18.0,296.3,680.8
281,意尔康品牌店,114452.0,65,7.7,229.8,271.5
161,乙方乙方88888,88144.0,9,60.3,162.3,164.7
197,吸引力xl,43870.0,5,59.0,148.7,152.0


In [8]:
df_te_zheng.set_index("nick",inplace=True)

In [11]:
df_te_zheng

,z_xse,z_num,p_sales,p_bdj,p_price
nick,,,,,
意尔康皮鞋旗舰店,3558446.0,289,47.8,257.4,485.7
意尔康男鞋旗舰店,792685.0,149,18.0,296.3,680.8
意尔康品牌店,114452.0,65,7.7,229.8,271.5
乙方乙方88888,88144.0,9,60.3,162.3,164.7
吸引力xl,43870.0,5,59.0,148.7,152.0
...,...,...,...,...,...
yek108,0.0,3,0.0,0.0,398.7
热血无极限,0.0,2,0.0,0.0,148.9
燕轩品牌店,0.0,1,0.0,0.0,178.0


In [12]:
df_te_zheng.loc["意尔康皮鞋旗舰店"]

z_xse      3558446.0
z_num          289.0
p_sales         47.8
p_bdj          257.4
p_price        485.7
Name: 意尔康皮鞋旗舰店, dtype: float64

## 布局组件与echarts的兼容问题

https://github.com/andfanilo/streamlit-echarts/issues/43

需要在程序开头加入如下代码：
    
并且所有echarts图片必须以函数的形式引入

In [ ]:
#st.markdown(" <style>iframe{ height: 300px !important } ", unsafe_allow_html=True)

* 如下代码是可以正常运行的

In [14]:
import streamlit as st
from streamlit_echarts import st_echarts
from pyecharts import options as opts
from pyecharts.charts import Bar
from streamlit_echarts import st_pyecharts
import pandas as pd
import numpy as np

#下面这行程序必须写入
st.markdown(" <style>iframe{ height: 300px !important } ", unsafe_allow_html=True)


a=pd.read_csv(r"D:\try\shoes.csv")

a_xm=a.groupby("info.鞋面材质").size()

def echart_shoes():
    return {
        "color":"red",
        "tooltip": {
          "trigger": 'axis',
          "axisPointer": {
            "type": 'shadow'
          }
        },
        "xAxis": {
            "type": "category",
            "data": a_xm.index.tolist(),
        },
        "yAxis": {"type": "value"},
        "series": [
            {"data": a_xm.values.tolist(), "type": "line"}
        ],
    }

def echarts_random_options():
    return {
        "xAxis": {"type": "category", "data": ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]},
        "yAxis": {"type": "value"},
        "series": [
            {"data": list(np.random.random(size=7) * 800),
             "type": "line"}
        ],
    }

    
left_column, right_column = st.columns(2)

with left_column:
    st_echarts(options=echart_shoes())


with right_column:
    st_echarts(options=echarts_random_options())

2023-03-14 19:47:47.644 
  command:

    streamlit run G:\anaconda\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


### classwork 2

* 在echarts官网找两张图并列输出试试

* 拷贝代码完成店铺特征数据的输出，利用多选框选择不同的店铺，输出对应的数据框

st.sidebar.multiselect

* 基于选择后的数据框做雷达图，并与上题的数据框并列输出

https://echarts.apache.org/examples/zh/editor.html?c=radar

## echarts事件与图表下钻在streamlit中的实现

https://github.com/andfanilo/streamlit-echarts

* echarts的事件与js函数
* echarts的隐含参数params

In [ ]:
option = {
    "xAxis": {
        "type": "category",
        "data": ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"],
    },
    "yAxis": { "type": "value" },
    "series": [
        {"data": [820, 932, 901, 934, 1290, 1330, 1320], "type": "line" }
    ],
}
events = {
    "click": "function(params) { console.log(params.name); return params.name }",
    "dblclick":"function(params) { return [params.type, params.name, params.value] }"
}
value = st_echarts(option, events=events)
st.write(value)  # shows name on bar click and type+name+value on bar double click

### classwork3

* 导入男鞋数据，基于“info.鞋面材质”groupby，求此列的商品数量，并作柱图，并且点击柱子输出对应的数据

* 点击上图柱子，输出对应材质的男鞋价格分布（注意这时双数值轴，要固定x轴范围）

https://echarts.apache.org/examples/zh/editor.html?c=line-in-cartesian-coordinate-system

In [6]:
a1.columns

Index(['_id.$oid', 'info.上市年份季节', 'info.上市时间', 'info.产品名称', 'info.低帮鞋品名',
       'info.功能', 'info.吊牌价', 'info.品牌', 'info.图案', 'info.场合', 'info.外底材料',
       'info.季节', 'info.尺码', 'info.帮面内里材质', 'info.帮面材质', 'info.性别',
       'info.是否商场同款', 'info.是否瑕疵', 'info.款号', 'info.款式', 'info.流行元素',
       'info.真皮材质工艺', 'info.细分风格', 'info.货号', 'info.跟底款式', 'info.运动系列',
       'info.运动鞋科技', 'info.适合路面', 'info.适用对象', 'info.销售渠道类型', 'info.闭合方式',
       'info.靴子品名', 'info.靴筒内里材质', 'info.靴筒材质', 'info.靴筒高度', 'info.鞋制作工艺',
       'info.鞋垫材质', 'info.鞋头款式', 'info.鞋底材质', 'info.鞋码', 'info.鞋跟高',
       'info.鞋面内里材质', 'info.鞋面材质', 'info.颜色分类', 'info.风格', 'itemid',
       'location', 'nick', 'price', 'sales', 'title', 'url', 'xse'],
      dtype='object')

In [3]:
jiage=a1.groupby("price").size()
jiage

price
16.9      1
17.9      1
18.0      3
25.0      1
58.0      4
         ..
2399.0    4
2999.0    2
3099.0    4
3499.0    4
6999.0    4
Length: 383, dtype: int64

In [7]:
list(jiage.items())

[(16.9, 1),
 (17.9, 1),
 (18.0, 3),
 (25.0, 1),
 (58.0, 4),
 (59.0, 1),
 (69.0, 3),
 (88.0, 2),
 (89.0, 4),
 (97.02, 1),
 (98.0, 5),
 (99.0, 17),
 (105.0, 1),
 (106.82, 1),
 (108.0, 2),
 (109.0, 7),
 (111.0, 1),
 (115.0, 7),
 (118.0, 31),
 (118.72, 1),
 (118.99, 1),
 (119.0, 4),
 (119.9, 2),
 (120.0, 1),
 (120.12, 1),
 (123.0, 2),
 (124.12, 1),
 (125.0, 1),
 (125.44, 1),
 (126.0, 1),
 (127.0, 2),
 (127.9, 1),
 (128.0, 82),
 (129.0, 12),
 (129.8, 1),
 (130.0, 1),
 (130.8, 1),
 (131.0, 1),
 (132.0, 4),
 (133.0, 8),
 (135.0, 7),
 (135.24, 1),
 (136.0, 1),
 (136.22, 1),
 (137.0, 10),
 (137.4, 1),
 (137.9, 1),
 (138.0, 38),
 (139.0, 30),
 (139.3, 1),
 (140.0, 6),
 (141.0, 2),
 (142.0, 3),
 (142.2, 1),
 (142.8, 1),
 (143.0, 8),
 (143.4, 1),
 (144.0, 1),
 (145.0, 8),
 (146.0, 24),
 (147.0, 5),
 (148.0, 103),
 (149.0, 39),
 (149.8, 1),
 (150.0, 15),
 (150.12, 2),
 (151.0, 1),
 (152.0, 8),
 (153.0, 5),
 (154.8, 1),
 (154.84, 3),
 (155.0, 5),
 (155.82, 2),
 (156.0, 20),
 (157.0, 2),
 (158.0, 160

In [7]:
a1.groupby("info.鞋面材质").size()

info.鞋面材质
PU             671
二层牛皮（除牛反绒）     605
二层猪皮             8
人造革             80
塑胶              10
多种材质拼接          21
太空革              9
头层牛皮（除牛反绒）    3470
头层猪皮             6
布                8
棉布               1
牛仔布              2
牛反绒             12
磨砂皮             14
绒面              15
网布               7
羊皮毛一体            1
超纤              35
超纤皮              1
鳄鱼皮              4
dtype: int64

In [10]:
x1=a[a["info.鞋面材质"]=="PU"].groupby("price").size()

In [20]:
data=[]
for i in x1.items():
    data.append(list(i))

In [22]:
        "xAxis": {
            "min":0,
            "max":600
        },

## 地图

https://github.com/andfanilo/streamlit-echarts-demo

* json格式的地图及其导入
* 地图数据
* jscode


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 20 18:21:36 2023

@author: wp
"""

import json
from streamlit_echarts import Map
from streamlit_echarts import JsCode
from streamlit_echarts import st_echarts


def render_usa():
    formatter = JsCode(
        "function (params) {"
        + "var value = (params.value + '').split('.');"
        + "value = value[0].replace(/(\d{1,3})(?=(?:\d{3})+(?!\d))/g, '$1,');"
        + "return params.seriesName + '<br/>' + params.name + ': ' + value;}"
    ).js_code

    with open(r"F:\notebooks1\streamlit\USA.json", "r") as f:
        map = Map(
            "USA",
            json.loads(f.read()),
            {
                "Alaska": {"left": -131, "top": 25, "width": 15},
                "Hawaii": {"left": -110, "top": 28, "width": 5},
                "Puerto Rico": {"left": -76, "top": 26, "width": 2},
            },
        )
    options = {
        "title": {
            "text": "USA Population Estimates (2012)",
            "subtext": "Data from www.census.gov",
            "sublink": "http://www.census.gov/popest/data/datasets.html",
            "left": "right",
        },
        "tooltip": {
            "trigger": "item",
            "showDelay": 0,
            "transitionDuration": 0.2,
            "formatter": formatter,
        },
        "visualMap": {
            "left": "right",
            "min": 500000,
            "max": 38000000,
            "inRange": {
                "color": [
                    "#313695",
                    "#4575b4",
                    "#74add1",
                    "#abd9e9",
                    "#e0f3f8",
                    "#ffffbf",
                    "#fee090",
                    "#fdae61",
                    "#f46d43",
                    "#d73027",
                    "#a50026",
                ]
            },
            "text": ["High", "Low"],
            "calculable": True,
        },
        "toolbox": {
            "show": True,
            "left": "left",
            "top": "top",
            "feature": {
                "dataView": {"readOnly": False},
                "restore": {},
                "saveAsImage": {},
            },
        },
        "series": [
            {
                "name": "USA PopEstimates",
                "type": "map",
                "roam": True,
                "map": "USA",
                "emphasis": {"label": {"show": True}},
                "textFixed": {"Alaska": [20, -20]},
                "data": [
                    {"name": "Alabama", "value": 4822023},
                    {"name": "Alaska", "value": 731449},
                    {"name": "Arizona", "value": 6553255},
                    {"name": "Arkansas", "value": 2949131},
                    {"name": "California", "value": 38041430},
                    {"name": "Colorado", "value": 5187582},
                    {"name": "Connecticut", "value": 3590347},
                    {"name": "Delaware", "value": 917092},
                    {"name": "District of Columbia", "value": 632323},
                    {"name": "Florida", "value": 19317568},
                    {"name": "Georgia", "value": 9919945},
                    {"name": "Hawaii", "value": 1392313},
                    {"name": "Idaho", "value": 1595728},
                    {"name": "Illinois", "value": 12875255},
                    {"name": "Indiana", "value": 6537334},
                    {"name": "Iowa", "value": 3074186},
                    {"name": "Kansas", "value": 2885905},
                    {"name": "Kentucky", "value": 4380415},
                    {"name": "Louisiana", "value": 4601893},
                    {"name": "Maine", "value": 1329192},
                    {"name": "Maryland", "value": 5884563},
                    {"name": "Massachusetts", "value": 6646144},
                    {"name": "Michigan", "value": 9883360},
                    {"name": "Minnesota", "value": 5379139},
                    {"name": "Mississippi", "value": 2984926},
                    {"name": "Missouri", "value": 6021988},
                    {"name": "Montana", "value": 1005141},
                    {"name": "Nebraska", "value": 1855525},
                    {"name": "Nevada", "value": 2758931},
                    {"name": "New Hampshire", "value": 1320718},
                    {"name": "New Jersey", "value": 8864590},
                    {"name": "New Mexico", "value": 2085538},
                    {"name": "New York", "value": 19570261},
                    {"name": "North Carolina", "value": 9752073},
                    {"name": "North Dakota", "value": 699628},
                    {"name": "Ohio", "value": 11544225},
                    {"name": "Oklahoma", "value": 3814820},
                    {"name": "Oregon", "value": 3899353},
                    {"name": "Pennsylvania", "value": 12763536},
                    {"name": "Rhode Island", "value": 1050292},
                    {"name": "South Carolina", "value": 4723723},
                    {"name": "South Dakota", "value": 833354},
                    {"name": "Tennessee", "value": 6456243},
                    {"name": "Texas", "value": 26059203},
                    {"name": "Utah", "value": 2855287},
                    {"name": "Vermont", "value": 626011},
                    {"name": "Virginia", "value": 8185867},
                    {"name": "Washington", "value": 6897012},
                    {"name": "West Virginia", "value": 1855413},
                    {"name": "Wisconsin", "value": 5726398},
                    {"name": "Wyoming", "value": 576412},
                    {"name": "Puerto Rico", "value": 3667084},
                ],
            }
        ],
    }
    st_echarts(options, map=map)


render_usa()


### classwork 4（地图在页面的展示）

* 简化代码，导入中国地图，并显示引入一个简单数据然后展示（encoding="utf-8"）

* 求每个省份的商品数量，并在地图中显示出来

In [12]:
location=a.location.str.split(" ",expand=True)
a["province"]=location[0]
a["city"]=location[1]

In [16]:
data0=[]
for i in a.groupby("province").size().items():
    data0.append({"name":i[0],"value":i[1]})
print(data0)

('上海', 154)
('北京', 339)
('吉林', 1)
('四川', 4)
('安徽', 24)
('山东', 44)
('山西', 39)
('广东', 284)
('江苏', 120)
('江西', 42)
('河北', 86)
('河南', 393)
('浙江', 2470)
('湖北', 27)
('福建', 952)
('贵州', 2)
('辽宁', 3)
('重庆', 3)
[{'name': '上海', 'value': 154}, {'name': '北京', 'value': 339}, {'name': '吉林', 'value': 1}, {'name': '四川', 'value': 4}, {'name': '安徽', 'value': 24}, {'name': '山东', 'value': 44}, {'name': '山西', 'value': 39}, {'name': '广东', 'value': 284}, {'name': '江苏', 'value': 120}, {'name': '江西', 'value': 42}, {'name': '河北', 'value': 86}, {'name': '河南', 'value': 393}, {'name': '浙江', 'value': 2470}, {'name': '湖北', 'value': 27}, {'name': '福建', 'value': 952}, {'name': '贵州', 'value': 2}, {'name': '辽宁', 'value': 3}, {'name': '重庆', 'value': 3}]


In [7]:
location=a.location.str.split(" ",expand=True)

In [8]:
a["province"]=location[0]
a["city"]=location[1]

In [18]:
data=[]
for i in a.groupby("province").size().items():
    data.append({"name":i[0],"value":i[1]})

In [19]:
data

[{'name': '上海', 'value': 154},
 {'name': '北京', 'value': 339},
 {'name': '吉林', 'value': 1},
 {'name': '四川', 'value': 4},
 {'name': '安徽', 'value': 24},
 {'name': '山东', 'value': 44},
 {'name': '山西', 'value': 39},
 {'name': '广东', 'value': 284},
 {'name': '江苏', 'value': 120},
 {'name': '江西', 'value': 42},
 {'name': '河北', 'value': 86},
 {'name': '河南', 'value': 393},
 {'name': '浙江', 'value': 2470},
 {'name': '湖北', 'value': 27},
 {'name': '福建', 'value': 952},
 {'name': '贵州', 'value': 2},
 {'name': '辽宁', 'value': 3},
 {'name': '重庆', 'value': 3}]

In [14]:
#list(zip([1,2],[3,4]))

In [10]:
#list(a.location.str.split(" ").items())

### classwork5（地图的下钻）

* 加入鼠标弹窗，显示省名与数值
* 做出浙江省的商品数量地图
* 在中国地图中加入点击事件，点击对应省份则展示对应省份的地图商品数